<a href="https://colab.research.google.com/github/kpsalida/HospitalDischarges_Visualization/blob/Katerina/viz_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Dataset**

The analysis utilizes the 2019 Hospital Inpatient Discharges (SPARCS De-Identified) dataset, publicly available through the New York State Open Data Portal (https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/4ny4-j5zv).  

SPARCS (Statewide Planning and Research Cooperative System) is a comprehensive administrative database that captures patient-level inpatient discharge records across New York State. The dataset includes clinical, operational, and financial variables such as length of stay (LOS), total charges, payer type, severity of illness, risk of mortality, and facility identifiers.

The dataset was restricted to discharges occurring in calendar year 2019 to avoid analysis inconsistencies arising from COVID-19 pandemic. Additionally it was narrowd to 3 Hospitals  Furthermore, the scope of the study was narrowed to  allow comparisons cross - facilities but also to maintain computational efficiency within the PowerBI environment. Two of the hospitals are major institutions in NYC while the third is more regional with a possibility of different LOS patterns and a different cost structure

With this filtering we are hoping for a more standardized performance comparison using normalized key performance indicators (KPIs), such as average LOS and average charges per discharge, independent of hospital size.

In [27]:
df.columns

Index(['Hospital Service Area', 'Hospital County',
       'Operating Certificate Number', 'Permanent Facility Id',
       'Facility Name', 'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'Type of Admission',
       'Patient Disposition', 'Discharge Year', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'CCSR Procedure Code',
       'CCSR Procedure Description', 'APR DRG Code', 'APR DRG Description',
       'APR MDC Code', 'APR MDC Description', 'APR Severity of Illness Code',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3', 'Birth Weight',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs'],
      dtype='object')

# Column Selection

We excluded deep medical analysis with CCSR terms
All ids that might be confusing.  
Discharge year as our dataset is related only to 2019  
#
We selected the following features according to business logic:
#

**Identification**  
Hospital County  
Facility Name  

**Demographics**  
Age Group  
Gender  
Race / Ethnicity (only if you want social analysis — otherwise skip)    
  
**OPERATIONS**  
Length of Stay (IMPORTANT → convert to numeric)  
Type of Admission  
Emergency Department Indicator  
'Patient Disposition'
  
**FINANCIAL**  
Total Charges (convert to numeric)  
Total Costs (convert to numeric)  

**CLINICAL COMPLEXITY**  
APR Severity of Illness Description

APR Risk of Mortality

APR DRG Description      

**PAYER**

Payment Typology 1 (KEEP ONLY THIS)


**About Low Variance Features**

In [9]:
# Import necessary libraries
import pandas as pd              # Data manipulation and analysis
import numpy as np               # Numerical operations, arrays, and math functions
import matplotlib.pyplot as plt  # Basic plotting library
import seaborn as sns            # Advanced statistical visualizations built on matplotlib

# Set pandas display option to show up to 100 columns when printing a DataFrame
pd.set_option('display.max_columns', 100)

# Load the colab drive
from google.colab import drive
import os

Mount Google Drive to access files.

In [4]:
# Mounts the drive; if already mounted, it just continues
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Jump directly into your specific project folder
%cd /content/drive/MyDrive/Colab Notebooks/FinalProject_Visualizatoin

/content/drive/MyDrive/Colab Notebooks/FinalProject_Visualizatoin


In [13]:
df = pd.read_csv('raw_hospital_discharges_2019.csv', low_memory = False)

In [14]:
df.head()

,Hospital Service Area,Hospital County,Operating Certificate Number,Permanent Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,Discharge Year,CCSR Diagnosis Code,CCSR Diagnosis Description,CCSR Procedure Code,CCSR Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Emergency Department Indicator,Total Charges,Total Costs
0,New York City,Kings,7001020,1305,Maimonides Medical Center,50 to 69,112,F,Other Race,Not Span/Hispanic,21,Urgent,Skilled Nursing Home,2019,INJ033,"COMPLICATION OF CARDIOVASCULAR DEVICE, IMPLANT...",CAR022,HEART VALVE REPLACEMENT AND OTHER VALVE PROCED...,162.0,CARDIAC VALVE PROCEDURES W AMI OR COMPLEX PDX,5.0,DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,4.0,Extreme,Extreme,Surgical,Medicaid,Medicaid,Self-Pay,NaN,False,"$401,807.05","$100,294.11"
1,New York City,Kings,7001020,1305,Maimonides Medical Center,0 to 17,112,F,Black/African American,Not Span/Hispanic,3,Emergency,Home or Self Care,2019,RSP009,ASTHMA,ESA004,NON-INVASIVE VENTILATION,141.0,ASTHMA,4.0,DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,3.0,Major,Moderate,Medical,Medicaid,Medicaid,Self-Pay,NaN,True,"$46,725.01","$23,058.16"
2,New York City,Kings,7001020,1305,Maimonides Medical Center,0 to 17,112,F,White,Unknown,1,Newborn,Home or Self Care,2019,PNL001,LIVEBORN,NaN,NaN,640.0,"NEONATE BIRTHWT >2499G, NORMAL NEWBORN OR NEON...",15.0,NEWBORNS AND OTHER NEONATES WITH CONDITIONS OR...,1.0,Minor,Minor,Medical,Medicaid,Medicaid,Self-Pay,04100,False,"$3,393",$490.45
3,New York City,Kings,7001020,1305,Maimonides Medical Center,18 to 29,112,F,White,Unknown,2,Urgent,Home or Self Care,2019,PRG023,COMPLICATIONS SPECIFIED DURING CHILDBIRTH,PGN002,SPONTANEOUS VAGINAL DELIVERY,560.0,VAGINAL DELIVERY,14.0,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1.0,Minor,Minor,Medical,Medicaid,Medicaid,Self-Pay,NaN,True,"$20,858.02","$5,445.28"
4,New York City,Kings,7001020,1305,Maimonides Medical Center,18 to 29,116,F,White,Not Span/Hispanic,2,Urgent,Home or Self Care,2019,PRG029,"UNCOMPLICATED PREGNANCY, DELIVERY OR PUERPERIUM",PGN004,ASSISTED VAGINAL DELIVERY,560.0,VAGINAL DELIVERY,14.0,"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1.0,Minor,Minor,Medical,Medicaid,Medicaid,Self-Pay,NaN,False,"$21,116.01","$5,295.36"


In [25]:
df.shape

(53390, 33)

In [16]:
df.columns

Index(['Hospital Service Area', 'Hospital County',
       'Operating Certificate Number', 'Permanent Facility Id',
       'Facility Name', 'Age Group', 'Zip Code - 3 digits', 'Gender', 'Race',
       'Ethnicity', 'Length of Stay', 'Type of Admission',
       'Patient Disposition', 'Discharge Year', 'CCSR Diagnosis Code',
       'CCSR Diagnosis Description', 'CCSR Procedure Code',
       'CCSR Procedure Description', 'APR DRG Code', 'APR DRG Description',
       'APR MDC Code', 'APR MDC Description', 'APR Severity of Illness Code',
       'APR Severity of Illness Description', 'APR Risk of Mortality',
       'APR Medical Surgical Description', 'Payment Typology 1',
       'Payment Typology 2', 'Payment Typology 3', 'Birth Weight',
       'Emergency Department Indicator', 'Total Charges', 'Total Costs'],
      dtype='object')

In [28]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Operating Certificate Number,53390.0,6.748851e+06,461858.850345,5901000.0,7001020.0,7001020.0,7001020.0,7001020.0
Permanent Facility Id,53390.0,1.248337e+03,104.624142,1039.0,1305.0,1305.0,1305.0,1305.0
Discharge Year,53390.0,2.019000e+03,0.000000,2019.0,2019.0,2019.0,2019.0,2019.0
APR DRG Code,53377.0,4.372920e+02,229.146079,4.0,204.0,469.0,640.0,956.0
APR MDC Code,53377.0,1.082491e+01,5.516998,0.0,5.0,11.0,15.0,25.0
APR Severity of Illness Code,53377.0,2.036214e+00,1.006992,0.0,1.0,2.0,3.0,4.0


In [23]:
df.value_counts('Facility Name',dropna=False)

,count
Facility Name,
Maimonides Medical Center,41129
NewYork-Presbyterian/Hudson Valley Hospital,8258
St. Joseph's Medical Center,4003


In [30]:
df.value_counts('Permanent Facility Id',dropna=False)

,count
Permanent Facility Id,
1305,41129
1039,8258
1098,4003


In [21]:
df.value_counts('Payment Typology 3', dropna=False)

,count
Payment Typology 3,
Self-Pay,28722
NaN,20260
Medicaid,3977
Private Health Insurance,241
Blue Cross/Blue Shield,113
Medicare,49
Federal/State/Local/VA,28


In [31]:
df.value_counts('Type of Admission', dropna=False)

,count
Type of Admission,
Emergency,32419
Urgent,9117
Newborn,8338
Elective,3508
Not Available,8


In [32]:
df.value_counts('Emergency Department Indicator',dropna=False)

,count
Emergency Department Indicator,
True,30744
False,22646


In [33]:
df.value_counts('Patient Disposition',dropna=False)

,count
Patient Disposition,
Home or Self Care,36907
Skilled Nursing Home,6696
Home w/ Home Health Services,5831
Expired,1258
Left Against Medical Advice,944
Short-term Hospital,748
Inpatient Rehabilitation Facility,308
Hospice - Home,205
Hospice - Medical Facility,173


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53390 entries, 0 to 53389
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Hospital Service Area                53390 non-null  object 
 1   Hospital County                      53390 non-null  object 
 2   Operating Certificate Number         53390 non-null  int64  
 3   Permanent Facility Id                53390 non-null  int64  
 4   Facility Name                        53390 non-null  object 
 5   Age Group                            53390 non-null  object 
 6   Zip Code - 3 digits                  52821 non-null  object 
 7   Gender                               53390 non-null  object 
 8   Race                                 53390 non-null  object 
 9   Ethnicity                            53390 non-null  object 
 10  Length of Stay                       53390 non-null  object 
 11  Type of Admission           